## Import libraries

In [1]:
import cassandra
from cassandra.cluster import Cluster

## Create a connection to the database

In [2]:
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

## Create a keyspace to do the work in and connect to it

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_keyspace
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    session.set_keyspace("udacity_keyspace")
except Exception as e:
    print(e)

## Music library

We need to create a new music library of albums.  

Here is the information asked of the data:  
Query: Give every album in the music library that was created by a given artist  
`select * from music_library WHERE artist_name="The Beatles"`  

Here is the collection of data:
![music_library_cassandra_db](images/music_library_cassandra_db_2.png)

## Data modeling

### Let's try by making the PRIMARY KEY only one column

In [4]:
# Create table
create_music_library = """
CREATE TABLE IF NOT EXISTS music_library (
    year INT,
    artist_name TEXT,
    album_name TEXT,
    city TEXT,
    PRIMARY KEY (artist_name)
);
"""

try:
    session.execute(create_music_library)
except Exception as e:
    print(e)
    
# Insert data
insert_music_library = """
INSERT INTO music_library (year, artist_name, album_name, city)
VALUES (%s, %s, %s, %s);
"""

music_library_data = [
    (1965, "The Beatles", "Rubber Soul", "Oxford"),
    (1970, "The Beatles", "Let it Be", "Liverpool"),
    (1966, "The Monkees", "The Monkees", "Los Angeles"),
    (1970, "The Carpenters", "Close To You", "San Diego"),
    (1965, "The Who", "My Generation", "London")
]

try:
    for item in music_library_data:
        session.execute(insert_music_library, item)
except Exception as e:
    print(e)

### Let's validate our data model. Does it give us two rows?

In [5]:
query = """
SELECT *
FROM music_library
WHERE artist_name = 'The Beatles';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

1970 The Beatles Let it Be Liverpool


It doesn't give us the two rows because we haven't created a UNIQUE PRIMARY KEY.

## Let's now use a COMPOSITE KEY made by two columns

In [6]:
# Create table
create_music_library = """
CREATE TABLE IF NOT EXISTS music_library2 (
    year INT,
    artist_name TEXT,
    album_name TEXT,
    city TEXT,
    PRIMARY KEY (artist_name, year)
);
"""

try:
    session.execute(create_music_library)
except Exception as e:
    print(e)
    
# Insert data
insert_music_library = """
INSERT INTO music_library2 (year, artist_name, album_name, city)
VALUES (%s, %s, %s, %s);
"""

music_library_data = [
    (1965, "The Beatles", "Rubber Soul", "Oxford"),
    (1970, "The Beatles", "Let it Be", "Liverpool"),
    (1966, "The Monkees", "The Monkees", "Los Angeles"),
    (1970, "The Carpenters", "Close To You", "San Diego"),
    (1965, "The Who", "My Generation", "London")
]

try:
    for item in music_library_data:
        session.execute(insert_music_library, item)
except Exception as e:
    print(e)

### Validate our new data model

In [7]:
query = """
SELECT *
FROM music_library2
WHERE artist_name = 'The Beatles';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

1965 The Beatles Rubber Soul Oxford
1970 The Beatles Let it Be Liverpool


Now we've got the expected output, so the data model is validated.

## Drop the tables

In [8]:
try:
    session.execute("DROP TABLE music_library;")
    session.execute("DROP TABLE music_library2;")
except Exception as e:
    print(e)

## Close session and cluster connection

In [9]:
session.shutdown()
cluster.shutdown()